# recalculate_sflm_processed_features

This notebook pulls the Single Feature to Label Mapping (SFLM) Google sheet and reprocesses the `unprocessed_features` column and writes back the `processed_features`. 

This is necessary as when running the `pull_objects_labelling_sheet.ipynb`, it will process any features from the `manual_data_labelling` sheet, and check for duplicates between the newly processed output and the current SFLM using its [`label` , `unprocessed_features`, `processed_features` ] columns. 

If the processing function is changed between runs of `pull_objects_labelling_sheet.ipynb`, it will mistakenly say that all previously added labels are new.

The reprocess_sflm function will reprocess the SFLM google sheet, and update the sheet with newly processed features. If the changed row was active it will also change the `status` to `"analyst_action_needed"` and the `use_processed_features` to `False`. The analyst's action is to check if that newly processed feature should be used in its processed form, or if it should use the unprocessed feature. 


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import pprint

import pandas as pd

from phoenix.common import artifacts, utils
from phoenix.tag.labelling import run_params, sflm_processing


In [ ]:
utils.setup_notebook_output()
utils.setup_notebook_logging()

In [ ]:
# Parameters
# See phoenix/common/run_datetime.py expected format of parameter
RUN_DATETIME = None
TENANT_ID = "test"

# See phoenix/common/artifacts/registry_environment.py expected format of parameter
ARTIFACTS_ENVIRONMENT_KEY = "local"

OBJECT_TYPE = "facebook_posts"


In [ ]:
cur_run_params = run_params.recalculate_sflm.create(ARTIFACTS_ENVIRONMENT_KEY, TENANT_ID, RUN_DATETIME, OBJECT_TYPE)

In [ ]:
# Display params.
pprint.pprint(cur_run_params)

In [ ]:
google_client = artifacts.google_sheets.get_client()

In [ ]:
labelled_objects_df = artifacts.google_sheets.get(
    google_client, cur_run_params.tenant_folder_id, cur_run_params.spreadsheet_name, cur_run_params.worksheet_name
)

In [ ]:
labelled_objects_df

In [ ]:
reprocessed_sflm = sflm_processing.reprocess_sflm(labelled_objects_df)

In [ ]:
num_action_needed_rows = reprocessed_sflm[reprocessed_sflm["status"] == "analyst_action_needed"].shape[0]
print(f"{num_action_needed_rows} processed features for active row changed, please notify analyst that action is needed")

In [ ]:
artifacts.google_sheets.persist(
    google_client, 
    cur_run_params.tenant_folder_id, 
    cur_run_params.spreadsheet_name, 
    cur_run_params.worksheet_name, 
    reprocessed_sflm
)